In [5]:
#使用魔法命令 %cd 更改成项目路径
#等价于
# import os
# os.chdir(r'~/Github/sspai_spider')
%cd ~/Github/sspai_spider/ 
import pandas as pd

/Users/linxiaoyue/Github/sspai_spider


# 首页文章数据预处理

In [6]:
#读入 data 文件夹中的数据
page = pd.read_excel(r'./data/page_data.xlsx') # Windows 用户需要修改路径
print(page.info())
page.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12243 entries, 0 to 12242
Data columns (total 26 columns):
advertisement_url      10 non-null object
author.avatar          12033 non-null object
author.id              12243 non-null int64
author.nickname        12243 non-null object
author.slug            12243 non-null object
banner                 11913 non-null object
comment_count          12243 non-null int64
corner.color           119 non-null object
corner.icon            357 non-null object
corner.id              12243 non-null int64
corner.memo            357 non-null object
corner.name            357 non-null object
corner.url             0 non-null float64
free                   12243 non-null bool
id                     12243 non-null int64
important              12243 non-null int64
like_count             12243 non-null int64
morning_paper_title    12243 non-null object
post_type              12243 non-null int64
released_time          12243 non-null datetime64[ns]
series

,advertisement_url,author.avatar,author.id,author.nickname,author.slug,banner,comment_count,corner.color,corner.icon,corner.id,...,like_count,morning_paper_title,post_type,released_time,series,special_columns,status,summary,title,view_count
0,NaN,attachment/origin/2015/01/06/211491.jpg,0,李小明,edxmd87l,attachment/origin/87/00/00/87.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 02:46:37,[],[],0,明天就是诺基亚正式在国内发布Windows Phone手机的日子，相信一定有很多WP党或者是...,诺基亚Lumia即将登陆，中国电信力推,2908
1,NaN,attachment/origin/2015/01/06/211491.jpg,0,李小明,edxmd87l,attachment/origin/90/00/00/90.jpg,0,NaN,NaN,0,...,2,[],1,2012-03-27 03:25:21,[],[],0,新 iPad 上拥有 500 万像素的背照式摄像头，摄影性能比上一代 iPad 要提高许多。...,Pix：iPad 上的拍照应用,4259
2,NaN,attachment/origin/2014/12/01/198940.jpg,0,laisoukin,82u1db40,attachment/origin/2015/02/12/225281.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 05:42:00,[],[],0,在使用iPhone时如何有电话打进来，会有“拒绝”和“接听”两个选项供选择。 但是在待机状态...,iPhone4S基础教程：待机状态如何实现挂断电话,24486
3,NaN,attachment/origin/2014/12/01/198940.jpg,0,laisoukin,82u1db40,attachment/origin/2015/02/12/225280.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 07:32:00,[],[],0,在如何添加/删除键盘这篇文章里，我们了解了如何在系统键盘中添加适合自己的键盘，来提高我们的输...,iPhone4S基础教程：如何快速切换键盘,13659
4,NaN,attachment/origin/2015/02/13/225448.jpg,0,地瓜,nc0ulnjo,attachment/origin/92/00/00/92.jpg,0,NaN,NaN,0,...,1,[],1,2012-03-27 08:16:37,[],[],0,"这次愤怒的小鸟以全新的姿态出现在了我们眼前,感觉更像一支战队.小鸟们遇到了一只外星鸟,外星鸟...",欢迎回来《愤怒的小鸟太空版》...,6113


字段选取与处理

In [7]:
page_cols = [
    'id', 'title', 'post_type', 'free',
    'author.slug', 'author.nickname', 'released_time',
    'comment_count', 'like_count', 'view_count'
]
page = page[page_cols]
page.columns = page.columns.str.replace('.', '_')
page['released_time'] = pd.to_datetime(page['released_time'])
page = (page
           .assign(released_date = page['released_time'].dt.date,
                      released_year = page['released_time'].dt.year,
                      released_month = page['released_time'].dt.month,
                      released_day = page['released_time'].dt.day,
                      released_at = page['released_time'].dt.time)
       )
page.head()

,id,title,post_type,free,author_slug,author_nickname,released_time,comment_count,like_count,view_count,released_date,released_year,released_month,released_day,released_at
0,1541,诺基亚Lumia即将登陆，中国电信力推,1,True,edxmd87l,李小明,2012-03-27 02:46:37,0,1,2908,2012-03-27,2012,3,27,02:46:37
1,1561,Pix：iPad 上的拍照应用,1,True,edxmd87l,李小明,2012-03-27 03:25:21,0,2,4259,2012-03-27,2012,3,27,03:25:21
2,1570,iPhone4S基础教程：待机状态如何实现挂断电话,1,True,82u1db40,laisoukin,2012-03-27 05:42:00,0,1,24486,2012-03-27,2012,3,27,05:42:00
3,1586,iPhone4S基础教程：如何快速切换键盘,1,True,82u1db40,laisoukin,2012-03-27 07:32:00,0,1,13659,2012-03-27,2012,3,27,07:32:00
4,1579,欢迎回来《愤怒的小鸟太空版》...,1,True,nc0ulnjo,地瓜,2012-03-27 08:16:37,0,1,6113,2012-03-27,2012,3,27,08:16:37


检查是否有缺失值

In [8]:
page.isnull().any()

id                 False
title              False
post_type          False
free               False
author_slug        False
author_nickname    False
released_time      False
comment_count      False
like_count         False
view_count         False
released_date      False
released_year      False
released_month     False
released_day       False
released_at        False
dtype: bool

## 首页文章数统计

In [ ]:
!pip install pyecharts #没有安装 pyecharts 的可以使用该命令安装

In [14]:
from pyecharts.charts import Line
from pyecharts import options as opts

seasonal_count = page.groupby(pd.Grouper(key='released_time', freq='M'))['title'].count().reset_index()

line = (
    Line()
    .add_xaxis(seasonal_count['released_time'].dt.date.tolist())
    .add_yaxis("", seasonal_count['title'].tolist(), is_smooth=True)
    .set_global_opts(datazoom_opts=opts.DataZoomOpts(type_='slider'), 
                            title_opts= opts.TitleOpts(title='少数派首页每月文章数统计（2012-2019）'))
)

line.render_notebook()

## 阅读量 TOP3 的文章

In [17]:
#被看最多的TOP3 文章
(page.groupby(['released_date', 'title', 'author_nickname'])['view_count']
 .sum()
 .sort_values(ascending=False)
 .to_frame()
 .head(3)
)

,,,view_count
released_date,title,author_nickname,
2014-04-16,认识与入门 Markdown,Te_Lee,1969521
2014-06-17,不止滤镜那么简单： VSCO Cam 上手指南,紫薯,1091499
2015-05-26,微信 Android 版你不知道的 13 个隐藏功能代码,壹啦罐罐,1045855


## 点赞/充电数 TOP3 的文章

In [18]:
#点赞数最多的 TOP3 文章
(page.groupby(['released_date', 'title', 'author_nickname'])['like_count']
 .sum()
 .sort_values(ascending=False)
 .to_frame()
 .head(3)
)

,,,like_count
released_date,title,author_nickname,
2019-02-21,40000 字、100 张配图，教你如何用 iPhone 拍出好照片 & 视频：iPhone 相机完全操作指南 2.0,果核Group,2087
2019-03-08,装了啥 | 看片、杀毒、效率办公……这 34 款 Windows 应用，每一款都是神器,Umi,1626
2019-03-28,告别伸手党，这 11 个网站让你有问题不求人,Umi,1508


## 评论数 TOP3 的文章

In [19]:
#被评论最多的文章
(page.groupby(['released_date', 'title', 'author_nickname'])['comment_count']
 .sum()
 .sort_values(ascending=False)
 .to_frame()
 .head(3)
)

,,,comment_count
released_date,title,author_nickname,
2017-10-23,不仅能快速查冰点限免，还能看到更多优秀 App 推荐：Price Tag 2.0,WATERS,428
2017-06-25,笔记 + 知识库 + 任务管理，数字笔记里的全能选手：Notion,Kralite,422
2019-08-28,和派聊聊 | 今年你买过最后悔的产品是什么？,Matrix机器人,372


## 综合排序TOP10

In [20]:
# 首页文章综合排序TOP10
(page.groupby(['released_date', 'title', 'author_nickname'])[['like_count','view_count', 'comment_count']]
 .sum()
 .sort_values(by=['like_count','view_count', 'comment_count'], ascending=False)
 .head(10)
)

,,,like_count,view_count,comment_count
released_date,title,author_nickname,,,
2019-02-21,40000 字、100 张配图，教你如何用 iPhone 拍出好照片 & 视频：iPhone 相机完全操作指南 2.0,果核Group,2087,131023,82
2019-03-08,装了啥 | 看片、杀毒、效率办公……这 34 款 Windows 应用，每一款都是神器,Umi,1626,85727,122
2019-03-28,告别伸手党，这 11 个网站让你有问题不求人,Umi,1508,74491,44
2017-11-22,这份 10000 字的 iPhone 相机指南，想告诉你原生相机就能拍出好照片,Derrick Zhang,1253,59411,46
2018-08-08,经过一年的思考，我重新梳理了我的印象笔记使用方法,曹将,1252,434695,76
2018-02-27,独居青年生活指南 | 2017 年度征文,卷毛的,1136,66804,122
2019-03-23,总是在重复「从入门到放弃 Python」？这里有为你「量身定做」的学习方法,玉树芝兰,1111,59693,41
2018-03-05,从六级刚过线到自学考出托福、GRE，这是我的方法总结 | 2017 年度征文,notajerk,1067,70673,103
2018-11-19,写给 Mac 新手的入门 App 指南,Vanilla,1015,86188,64


# 作者成就数据预处理

In [9]:
user = pd.read_excel(r'./data/user_data.xlsx') #同上
print(user.info())
user.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 679 entries, 0 to 678
Data columns (total 23 columns):
data.article_view_count       679 non-null int64
data.articles_word_count      679 non-null int64
data.avatar                   653 non-null object
data.avatar_id                679 non-null int64
data.bio                      538 non-null object
data.created_at               679 non-null int64
data.followed                 679 non-null bool
data.followed_count           679 non-null int64
data.following_count          679 non-null int64
data.id                       679 non-null int64
data.key                      653 non-null object
data.liked_count              679 non-null int64
data.nickname                 679 non-null object
data.power_plus_flag          679 non-null int64
data.slug                     679 non-null object
data.social_networks          679 non-null object
data.special_column_count     679 non-null int64
data.special_column_writer    679 non-null bool
data.user

,data.article_view_count,data.articles_word_count,data.avatar,data.avatar_id,data.bio,data.created_at,data.followed,data.followed_count,data.following_count,data.id,...,data.power_plus_flag,data.slug,data.social_networks,data.special_column_count,data.special_column_writer,data.user_badges,data.user_flags,error,data.occupation1,data.occupation2
0,43872,6463,attachment/origin/2015/01/06/211491.jpg,211491,妄图成为“技术帝”的无名小编，坚定的互联网从业者,1330906220,False,0,0,4,...,0,edxmd87l,[],0,False,[],[],0,NaN,NaN
1,4562816,137900,attachment/origin/2014/12/01/198940.jpg,198940,Je ne suis pas l'auteur,1330883844,False,2,0,3,...,0,82u1db40,[],0,False,[],[],0,NaN,NaN
2,258142,103329,attachment/origin/2015/02/13/225448.jpg,225448,搬砖无限好，我来砌长城。,1332698004,False,1,0,9,...,0,nc0ulnjo,[],0,False,[],[],0,NaN,NaN
3,774645,135141,attachment/origin/2014/12/23/207139.jpg,207139,凛冬将至！,1332716184,False,3,0,10,...,0,w5si9qrh,[],0,False,[],[],0,NaN,NaN
4,1023415,160287,2017/04/06/064a7973c2c467708092d7c8b423be9a.jpg,204472,少数派首席打杂，有任何建议随时私信我,1330906250,False,470,795,5,...,1,michael,"[{'id': 83728, 'user_id': 5, 'type': 1, 'nickn...",1,True,"[{'id': 4596, 'name': 'Power+ User', 'descript...","[{'id': 5209, 'name': '少数派成员', 'description': ...",0,少数派成员,NaN


字段选取

In [10]:
user_cols = [
    'data.slug', 'data.nickname', 'data.created_at',
    'data.article_view_count', 'data.articles_word_count', 'data.liked_count', 
    'data.followed_count', 'data.power_plus_flag', 'data.occupation1', 'data.occupation2'
]
user = user[user_cols]

字段重命名

In [11]:
user.columns = user.columns.str.extract(r'data.(.+)')[0].tolist()
user.rename(columns = {'slug': 'author_slug', 
                                   'nickname': 'author_nickname', 
                                   'article_view_count': 'view_count',
                                   'articles_word_count': 'word_count',
                                   'liked_count': 'like_count'}, 
                  inplace=True)

日期字段处理

In [12]:
user['created_at'] = pd.to_datetime(user['created_at'], unit='s')
user['today'] = pd.date_range('2019-10-01', '2019-10-01', periods=len(user))
user['sspai_days'] = (user['today'] - user['created_at']).dt.days
user.drop('today', axis=1, inplace=True)

## 少数派作者头衔统计

In [21]:
from pyecharts.charts import Pie
from pyecharts import options as opts

author_type = (pd.concat([user['occupation1'], user['occupation2']])
                         .value_counts()
                         .to_frame(name='count')
                         .reset_index()
                    )

pie = (
    Pie()
    .add('',author_type.values)
    .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}"))
)

pie.render_notebook()

## 累积阅读量 TOP5 作者、累积写作字数 TOP5 的作者、累积点赞/充电数 TOP5 

In [23]:
from pyecharts.charts import Funnel, Tab
from pyecharts import options as opts

def top5_author_funnel(col, label):
    _top5 = (user.groupby(['author_nickname'])[col]
                                   .sum()
                                   .sort_values(ascending=False)[:5]
                                   .to_dict()
            )
    
    _top5_funnel = (
        Funnel()
        .add('', _top5.items())
        .set_global_opts(title_opts=opts.TitleOpts(title='{}成就 TOP5'.format(label), 
                                                                  pos_left = 'center'),
                                legend_opts=opts.LegendOpts(is_show=False))
        .set_series_opts(label_opts=opts.LabelOpts(formatter="{b}: {c}", position='inside'))
    )
    
    return _top5_funnel


top5_view_funnel = top5_author_funnel('view_count', '阅读量')
top5_word_funnel = top5_author_funnel('word_count', '写作字数')
top5_like_funnel = top5_author_funnel('like_count', '点赞/充电数')

# combine charts
tab = (
    Tab()
    .add(top5_view_funnel, '阅读量')
    .add(top5_word_funnel, '写作字数')
    .add(top5_like_funnel, '点赞/充电数')

)

tab.render_notebook()

## 综合排序 TOP10 作者

In [25]:
from pyecharts.charts import Bar
from pyecharts import options as opts
from pyecharts.globals import ThemeType

user_top10_author = (user.groupby(['author_nickname'])[['view_count','like_count', 'word_count']]
                                    .sum()
                                    .sort_values(['view_count','like_count', 'word_count'], ascending=False)
                                    .head(10)
                             )

bar = (
    Bar(opts.InitOpts(height='450px', width='1600px', theme=ThemeType.LIGHT))
    .add_xaxis(user_top10_author.index.unique().tolist())
    .add_yaxis('阅读数', user_top10_author['view_count'].tolist())
    .add_yaxis('点赞/充电数', user_top10_author['like_count'].tolist())
    .add_yaxis('字数', user_top10_author['word_count'].tolist())
    .set_global_opts(title_opts = opts.TitleOpts(title='少数派作者成就综合排序', 
                                                                 subtitle='前十位', 
                                                                 pos_left='center'),
                            toolbox_opts= opts.ToolboxOpts(is_show=True), 
                            legend_opts= opts.LegendOpts(pos_right='20%'))
)

bar.render_notebook()

## TOP10 作者 2013 年-2019 年数据变化

In [28]:
from pyecharts import options as opts
from pyecharts.charts import Timeline


page_total_top10_df = (page.set_index('author_nickname')
                                        .loc[user_top10_author.index.tolist(),  \
                                             ['view_count', 'like_count', 'comment_count', 'released_year']]
                                        .sort_values(['released_year']))


def timeline_bar() -> Timeline:
    x = page_total_top10_df.index.unique().tolist()
    year = page_total_top10_df['released_year'].unique().tolist()
    tl = (Timeline()
          .add_schema(play_interval = 1000, 
                             is_timeline_show=True,  #将 True 改为 False 则隐藏播放条
                             is_auto_play = False) #这里改为 True 则会自动播放，但是会很卡（慎用）
         )
    for i in year:
        data = page_total_top10_df[page_total_top10_df['released_year'] == i]
        bar = (
            Bar()
            .add_xaxis(x)
            .add_yaxis("阅读量", data['view_count'].tolist())
            .add_yaxis("点赞/充电数", data['like_count'].tolist())
            .add_yaxis('评论数', data['comment_count'].tolist())
            .set_global_opts(title_opts=opts.TitleOpts("{}年各作者综合排序".format(i)), 
                                    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=15, 
                                                                                                                    font_size = 0.1, 
                                                                                                                    font_style='italic', 
                                                                                                                    horizontal_align = 'center', 
                                                                                                                    margin=25), 
                                                                        axistick_opts=opts.AxisTickOpts(is_align_with_label=True)),
                                    yaxis_opts=opts.AxisOpts(max_=100000))
        )
        tl.add(bar, "{}年".format(i))

    return tl

timeline_bar().render_notebook()